<a href="https://colab.research.google.com/github/jot240/Cassignment1/blob/master/DistrictHeating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pulp

In [28]:
from pulp import *
import pandas as pd
import numpy as np
from math import *

In [10]:
   def read_excel_data(filename, sheet_name):
      data = pd.read_excel(filename, sheet_name=sheet_name, header=None)
      values = data.values
      return values

In [11]:
file1 = "InputDataEnergySmallInstance.xlsx"
v0 = read_excel_data(file1,"SourceNum")#location of first node
vi = read_excel_data(file1,"Nodes") #number of nodes
v_cord = read_excel_data(file1,"NodesCord") # nodes coordinates
theta_ij_fix = read_excel_data(file1,"vfix(thetaijfix)")#Vfix cost
theta_ij_var = read_excel_data(file1,"vvar(thetaijvar)")#Vvar cost
fixed_unit_cost = read_excel_data(file1,"FixedUnitCost")#cost for 1 unit
c_heat = read_excel_data(file1,"cheat(ciheat)")# heat cost for verticie i
c_ij_var = read_excel_data(file1,"cvar(cijvar)")#variable cost for edge ij
c_ij_om = read_excel_data(file1,"com(cijom)")# operations+maintenence cost for edge ij
c_ij_rev = read_excel_data(file1,"crev(cijrev)") #revenue for edge ij
t_full_load_hours = read_excel_data(file1,"Tflh(Tiflh)") #operation time of source verticis
beta = read_excel_data(file1,"Betta")#concurrence effect parameter
connection_quota = read_excel_data(file1,"Lambda") #connection quota parameter (lamda in paper)
alpha = read_excel_data(file1,"Alpha")# annuity factor for investment costs
d_ij_peak = read_excel_data(file1,"EdgesDemandPeak(dij)")#peak demand for edges ij (kW)
d_ij_annual = read_excel_data(file1,"EdgesDemandAnnual(Dij)")#annual demand for edges ij (kWh/year)
cap_ij_max = read_excel_data(file1,"Cmax(cijmax)")#maximum pipe capacity for edge (kW)
cap_source_max = read_excel_data(file1,"SourceMaxCap(Qimax)")#(Qimax)Maximum heat generation capacity for source i (kW)
unmet_demand_penalty = read_excel_data(file1,"pumd(pijumd)")#(pumd)unmet demand penalty for edge ij


In [29]:
 #Create decision Variables
node_num = vi[0][0]
set_I = [*range(node_num)] #nodes
distances_ij = np.zeros(shape=(node_num,node_num))
print(v_cord[0][1])
for i in set_I:
  for j in set_I:
    distances_ij[i][j] = sqrt((v_cord[i][0]- v_cord[j][0])**2 + (v_cord[i][1]-v_cord[j][1])**2)

x_ij =  LpVariable.dicts('x', (set_I, set_I ),0,1, "Binary")
p_in_ij = LpVariable.dicts('pin', (set_I,set_I),lowBound=0)
p_out_ij = LpVariable.dicts('pin', (set_I,set_I),lowBound=0)






30
[[ 0.         14.14213562 28.28427125 14.14213562 31.6227766  40.
  31.6227766  28.28427125]
 [14.14213562  0.         14.14213562 20.         20.         31.6227766
  28.28427125 31.6227766 ]
 [28.28427125 14.14213562  0.         31.6227766  14.14213562 28.28427125
  31.6227766  40.        ]
 [14.14213562 20.         31.6227766   0.         28.28427125 31.6227766
  20.         14.14213562]
 [31.6227766  20.         14.14213562 28.28427125  0.         14.14213562
  20.         31.6227766 ]
 [40.         31.6227766  28.28427125 31.6227766  14.14213562  0.
  14.14213562 28.28427125]
 [31.6227766  28.28427125 31.6227766  20.         20.         14.14213562
   0.         14.14213562]
 [28.28427125 31.6227766  40.         14.14213562 31.6227766  28.28427125
  14.14213562  0.        ]]
{0: {0: x_0_0, 1: x_0_1, 2: x_0_2, 3: x_0_3, 4: x_0_4, 5: x_0_5, 6: x_0_6, 7: x_0_7}, 1: {0: x_1_0, 1: x_1_1, 2: x_1_2, 3: x_1_3, 4: x_1_4, 5: x_1_5, 6: x_1_6, 7: x_1_7}, 2: {0: x_2_0, 1: x_2_1, 2: x_2_2, 3

In [ ]:
print(x_ij)
#make problem
district_heating = LpProblem("district_heating")


# New Section